In [1]:
# t_loss_bkwd vs t_loss_bkwd2 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd, t_loss_bkwd2
BS, H, N, D = 2, 2, 5, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
#print(mask)

res2 = t_loss_bkwd(layers_params, y, mask, None, True, p_gen_aux)
#print(res2[1])
print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_loss_bkwd2(layers_params, y, mask, None, True, p_gen_aux)
#print(res3)
print_res_shapes(res3[0]) 


5
- 2
-- torch.Size([6, 4]) tensor([-0.054427,  0.153246, -0.024114], device='cuda:0')
-- torch.Size([6]) tensor([ 0.165028, -0.035873, -0.129454], device='cuda:0')
- 1
-- torch.Size([5, 4]) tensor([-0.028046, -0.044563, -0.020177], device='cuda:0')
- 10
-- torch.Size([4]) tensor([ 7.932237e-05, -1.188597e-04,  1.259867e-05], device='cuda:0')
-- torch.Size([4]) tensor([ 9.668623e-05,  1.931267e-04, -5.131257e-05], device='cuda:0')
-- torch.Size([2, 3, 2, 4]) tensor([ 0.003297, -0.003310, -0.001549], device='cuda:0')
-- torch.Size([4, 4]) tensor([ 0.000242, -0.000832, -0.001930], device='cuda:0')
-- torch.Size([4]) tensor([6.587836e-06, 5.283717e-05, 1.544247e-04], device='cuda:0')
-- torch.Size([4]) tensor([ 1.020024e-05, -8.035130e-05, -1.248818e-04], device='cuda:0')
-- torch.Size([16, 4]) tensor([-0.000737,  0.001317, -0.001313], device='cuda:0')
-- torch.Size([16]) tensor([ 0.000607, -0.000634, -0.001001], device='cuda:0')
-- torch.Size([4, 16]) tensor([-0.002221, -0.003061,  0.006

RuntimeError: shape '[16]' is invalid for input of size 160

In [ ]:
%env CUDA_VISIBLE_DEVICES=""

# t_avg_cross_entropy_loss_bkwd vs t_avg_cross_entropy_loss_bkwd2 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
from model_triton import t_gpt2_forward
from loss_and_optimizer_triton import t_avg_cross_entropy_loss_bkwd, t_avg_cross_entropy_loss_bkwd2
BS, H, N, D = 2, 2, 5, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cpu")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cpu")

y_in = y[:, :-1]
y_out = y[:, 1:]
logits = t_gpt2_forward(layers_params, y_in, mask, None, True, p_gen_aux) 

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
#print(mask)

res2 = t_avg_cross_entropy_loss_bkwd(y_out, logits)
print(res2.shape, res2)
#print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_avg_cross_entropy_loss_bkwd2(y_out, logits)
print(res3.shape, res3)
#print_res_shapes(res3[0]) 
